In [29]:
import gradio as gr
import joblib as jb
import numpy as np
import pandas as pd
import pickle as pk
import seaborn as sns
import sklearn.cluster as cluster
import warnings
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from IPython.display import Image
from kneed import KneeLocator
from matplotlib import pyplot as plt
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from pandas.core.dtypes.common import is_numeric_dtype
from sklearn import tree
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split as tts
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
warnings.filterwarnings("ignore")

In [30]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [31]:
df = pd.read_excel('data.xlsx')

In [32]:
Tfidf = TfidfVectorizer()

In [33]:
df.head()

,test,class
0,I love Bangladesh,1
1,Could you give me an iphone?,0
2,Hello how are you?,1
3,I want to talk you.,1


In [34]:
cv = CountVectorizer()

In [35]:
x = cv.fit_transform(df['test'])
x

<4x14 sparse matrix of type '<class 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>

In [36]:
x.toarray()

array([[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]], dtype=int64)

In [37]:
df.head()

,test,class
0,I love Bangladesh,1
1,Could you give me an iphone?,0
2,Hello how are you?,1
3,I want to talk you.,1


In [38]:
df3 = df.copy()

In [39]:
df2 = pd.DataFrame(x.toarray(), index = df['test'], columns = cv.get_feature_names_out())

In [40]:
df2.head()

,an,are,bangladesh,could,give,hello,how,iphone,love,me,talk,to,want,you
test,,,,,,,,,,,,,,
I love Bangladesh,0,0,1,0,0,0,0,0,1,0,0,0,0,0
Could you give me an iphone?,1,0,0,1,1,0,0,1,0,1,0,0,0,1
Hello how are you?,0,1,0,0,0,1,1,0,0,0,0,0,0,1
I want to talk you.,0,0,0,0,0,0,0,0,0,0,1,1,1,1


In [41]:
columns = cv.get_feature_names_out()
columns

array(['an', 'are', 'bangladesh', 'could', 'give', 'hello', 'how',
       'iphone', 'love', 'me', 'talk', 'to', 'want', 'you'], dtype=object)

# TF-IDF Vectorizer

In [42]:
idf = TfidfVectorizer()

In [43]:
x = idf.fit_transform(df3['test'])

In [44]:
x.toarray()

array([[0.        , 0.        , 0.70710678, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.70710678, 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.43003652, 0.        , 0.        , 0.43003652, 0.43003652,
        0.        , 0.        , 0.43003652, 0.        , 0.43003652,
        0.        , 0.        , 0.        , 0.27448674],
       [0.        , 0.5417361 , 0.        , 0.        , 0.        ,
        0.5417361 , 0.5417361 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.34578314],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.5417361 , 0.5417361 , 0.5417361 , 0.34578314]])

In [45]:
df4 = pd.DataFrame(x.toarray(), index = df['test'], columns = idf.get_feature_names_out())

In [46]:
df4.head()

,an,are,bangladesh,could,give,hello,how,iphone,love,me,talk,to,want,you
test,,,,,,,,,,,,,,
I love Bangladesh,0.000000,0.000000,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000
Could you give me an iphone?,0.430037,0.000000,0.000000,0.430037,0.430037,0.000000,0.000000,0.430037,0.000000,0.430037,0.000000,0.000000,0.000000,0.274487
Hello how are you?,0.000000,0.541736,0.000000,0.000000,0.000000,0.541736,0.541736,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.345783
I want to talk you.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.541736,0.541736,0.541736,0.345783


# Word2Vec

In [47]:
import gensim
from gensim.models import Word2Vec

In [48]:
from nltk.tokenize import sent_tokenize

In [51]:
text_vsc = [nltk.word_tokenize(test) for test in df['test']]
text_vsc

[['I', 'love', 'Bangladesh'],
 ['Could', 'you', 'give', 'me', 'an', 'iphone', '?'],
 ['Hello', 'how', 'are', 'you', '?'],
 ['I', 'want', 'to', 'talk', 'you', '.']]

In [54]:
text = []

In [55]:
for test in df['test']:
    text.append(nltk.word_tokenize(test))

In [56]:
text

[['I', 'love', 'Bangladesh'],
 ['Could', 'you', 'give', 'me', 'an', 'iphone', '?'],
 ['Hello', 'how', 'are', 'you', '?'],
 ['I', 'want', 'to', 'talk', 'you', '.']]

In [60]:
model = Word2Vec(text, min_count = 1)
model

In [61]:
model.wv.most_similar('Hello')

[('?', 0.17272651195526123),
 ('Bangladesh', 0.16695065796375275),
 ('give', 0.11118055880069733),
 ('talk', 0.10947782546281815),
 ('you', 0.07967711985111237),
 ('an', 0.04130832105875015),
 ('me', 0.037714049220085144),
 ('to', 0.0132435392588377),
 ('I', 0.008316040970385075),
 ('love', -0.005900950636714697)]